In [1]:
'''Load Libs'''
import numpy as np
import pandas as pd
import os.path
from pathlib import Path
from collections import Counter
import random
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from time import time

In [2]:
'''Load Data And Identify Classes'''
PathTrain =  os.getcwd() + "/data/train.csv"
PathTest =  os.getcwd() + "/data/test.csv"
DFTrain = pd.read_csv(PathTrain, header=None)
DFTest = pd.read_csv(PathTest, header=None)
totalClasses = len(Counter(DFTrain.iloc[:,-1]))

In [4]:
DTrain = DFTrain.iloc[:,:-1]
DTrain = np.c_[np.ones(len(DTrain)), DTrain]
LabelsTrain = np.array(DFTrain.iloc[:,-1])
DTest = DFTest.iloc[:,:-1]
DTest = np.c_[np.ones(len(DTest)), DTest]
LabelsTest = np.array(DFTest.iloc[:,-1])

In [5]:
'''One Hot Encoding for Train and Test Labels'''
oneHotTrainLables = np.zeros((len(DFTrain), totalClasses))

for i in range(len(DFTrain)):
    oneHotTrainLables[i][DFTrain.iloc[i, -1]] = 1

In [144]:
def sigmoid(z):
    #Sigmoid Function
    return 1/(1+np.exp(-z))

def ReLU(z):
    #Rectified Linear Unit
    return np.maximum(0,z)

def delSigmoid(z):
    #oj = sigmoid(z)
    return np.multiply(z, (1-z))

def delReLU(z):
    z = np.matrix(z)
    z[z > 0] = 1
    z[z <= 0] = 0    
    return z
    
    
class Layer:    
    
    def __init__(self, perceptron_units, n_inputs, activation_func, layer_type):
        self.type = layer_type
        self.activation = activation_func
        self.perceptron_units = perceptron_units
        self.inputs = None
        self.output = None
        #self.weights = 0.1*np.ones((n_inputs, perceptron_units))
        #self.weights = 0.0001*np.random.rand(n_inputs, perceptron_units)  
        #self.weights = np.random.normal(0,0.05, perceptron_units * (n_inputs)).reshape(n_inputs, perceptron_units)
        self.weights = np.random.normal(0,0.05, n_inputs * perceptron_units).reshape(n_inputs, perceptron_units)
        self.delta = None
        
    def __repr__(self):
        representation = (self.type, self.perceptron_units, self.weights.shape, self.activation)
        return "<%s Layer | Num_Perceptrons_Units = %d, Weights = %s, Activation = %s>" % representation
    

class NeuralNetwork:
    
    def __init__(self, list_hidden_layers, op_layers_activation, hidden_layers_activation):
        np.random.seed(525)
        self.total_layers = len(list_hidden_layers) + 1
        self.nodes_hidden_layers = list_hidden_layers
        self.layers = []
        
        for i in range(len(list_hidden_layers)):
            if i == 0:
                layer = Layer(list_hidden_layers[i], n, hidden_layers_activation, "Hidden")
                self.layers.append(layer)
            else:
                layer = Layer(list_hidden_layers[i], list_hidden_layers[i-1], hidden_layers_activation, "Hidden")                
                self.layers.append(layer)
        
        layer = Layer(r, list_hidden_layers[-1], op_layers_activation, "Output")
        self.layers.append(layer)
        
        
    def __repr__(self):
        layers = self.layers
        rep = ""
        print("Neural Network:")
        for i in range(len(layers)):
            rep += "Layer %d: %s\n" % (i, layers[i])
        return rep
    
    def forwardFeed(self, ip_data):
        layer = self.layers[0]
        layer.inputs = np.matrix(ip_data)
        layer.netj = np.matmul(layer.inputs, np.matrix(layer.weights))
        layer.output = self.activation(layer.netj, layer.activation)
        for i in range(1, len(self.layers), 1):
            layer = self.layers[i]
            layer.inputs = self.layers[i-1].output
            layer.netj = np.matmul(layer.inputs, np.matrix(layer.weights))
            layer.output = self.activation(layer.netj, layer.activation)
    
    def backwardPropagation(self, labels):
        output_layer = self.layers[-1]
        error = labels - output_layer.output
        delOj = self.delActivation(output_layer.output, output_layer.activation)
        output_layer.delta = -1*np.multiply(error, delOj)
        for i in reversed(range(self.total_layers-1)):
            current_layer = self.layers[i]
            next_layer = self.layers[i + 1]
            error = np.matmul(next_layer.delta, next_layer.weights.T)
            delOj = self.delActivation(current_layer.output, current_layer.activation)
            current_layer.delta = np.multiply(error, delOj)
            
    def activation(self, x, activation):
        if activation == "Sigmoid":
            return sigmoid(x)
        elif activation == "ReLU":
            return ReLU(x)
    
    def delActivation(self, x, activation):
        if activation == "Sigmoid":            
            return delSigmoid(x)
        
        elif activation == "ReLU":
            return delReLU(x)
        
    def updateParams(self, lr, bSize):
        layers = self.layers
        for layer in layers:
            #print(layer.inputs.shape, layer.delta.shape)
            gradient = np.matmul(layer.inputs.T, layer.delta)
            layer.weights = layer.weights - (lr/bSize)*gradient
    
            
    def meanSquaredError(self, Y, avg=True):
        div = 1
        #if avg:
        div = len(Y)
        
        op_layer_labels = self.layers[-1].output
        error = Y - op_layer_labels
        error = np.square(error)
        error = np.sum(error)/(2*div)
        return error
    
    
    def fit(self, X, Labels, eta = 0.1, batch_size=100, max_epoch = 10000, epsilon=1e-9, adaptive=False):
        lr = eta
        data = X
        labels = Labels
        
        epoch = 0
        self.forwardFeed(data)
        error_prev = self.meanSquaredError(labels)
        epoch_error_list = [error_prev]
        t0 = time()
        while epoch < max_epoch:
            t1 = time()
            epoch += 1
            
            data, labels = shuffle(data, labels)
            
            if adaptive:
                lr = eta/np.sqrt(epoch)
                
            for batch_start in range(0, len(data), batch_size):
                batch_end = batch_start + batch_size
                Xb = data[batch_start : batch_end]
                Yb = labels[batch_start : batch_end]
                
                self.forwardFeed(Xb)
                self.backwardPropagation(Yb)
                
                self.updateParams(lr, batch_size)
                #return
                
            t2 = time()   
            self.forwardFeed(data)
            error = self.meanSquaredError(labels)
            deltaError = error - error_prev
            epoch_error_list.append(np.abs(deltaError))            
            print("$$ Epoch: {} | Error = {} | DeltaError = {} | LR = {} | Epoch Train Time = {}Sec"
                  .format(epoch, round(error,6), round(deltaError,20), round(lr,5), round(t2-t1,2)))
            avg_deltaError = np.mean(epoch_error_list[-10:])
            if np.abs(avg_deltaError) < epsilon[1] or error < epsilon[0]:
                break
            error_prev = error
        
        t4 = time()
        print("\n%% Total Epochs ={} | Epsilon = {} | Total Learning Time = {}Min"
              .format(epoch, epsilon, round((t4-t0)/60,2)))
        
    def predict(self, data, labels):
        self.forwardFeed(data)
        outputs = self.layers[-1].output
        prediction = []
        for x in outputs:
            prediction.append(np.argmax(x))
        print(len(prediction), labels.shape)
        accu_score = accuracy_score(labels, prediction)
        #print("Accuracy Score: ", 100*accu_score)
        return 100*accu_score


In [145]:
'''Global Params'''
M = 100 #MiniBatch Size
n = len(DTrain[0])
r = totalClasses

In [138]:
'''Global Params'''
nodesInHiddenLayers = [100]
print("@@@@@@@@@@---------Q1_b---------@@@@@@@@@@")
print("######------HiddenLayer = {}-----######\n".format(nodesInHiddenLayers))
neu_net = NeuralNetwork(nodesInHiddenLayers, "Sigmoid", "Sigmoid")
print(neu_net)
neu_net.fit(DTrain, oneHotTrainLables, eta=0.1, max_epoch=10000, epsilon=[0.02, 1e-06])

AccuTrain = neu_net.predict(DTrain, LabelsTrain)
AccuTest = neu_net.predict(DTest, LabelsTest)
print("Train Accuracy = {}%".format(round(AccuTrain,3)))
print("Test Accuracy = {}%".format(round(AccuTest,3)))

@@@@@@@@@@---------Q1_b---------@@@@@@@@@@
######------HiddenLayer = [100]-----######

Neural Network:
Layer 0: <Hidden Layer | Num_Perceptrons_Units = 100, Weights = (785, 100), Activation = Sigmoid>
Layer 1: <Output Layer | Num_Perceptrons_Units = 26, Weights = (100, 26), Activation = Sigmoid>

$$ Epoch: 1 | Error = 0.478075 | DeltaError = -2.696344616666763 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Epoch: 2 | Error = 0.474252 | DeltaError = -0.0038234380173904903 | LR = 0.1 | Epoch Train Time = 0.2Sec
$$ Epoch: 3 | Error = 0.468909 | DeltaError = -0.005342356528371173 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 4 | Error = 0.460777 | DeltaError = -0.008132313595168084 | LR = 0.1 | Epoch Train Time = 0.22Sec
$$ Epoch: 5 | Error = 0.448158 | DeltaError = -0.012618476196315011 | LR = 0.1 | Epoch Train Time = 0.19Sec
$$ Epoch: 6 | Error = 0.433055 | DeltaError = -0.015103690745841836 | LR = 0.1 | Epoch Train Time = 0.21Sec
$$ Epoch: 7 | Error = 0.416789 | DeltaError = -0.01626561

$$ Epoch: 74 | Error = 0.108458 | DeltaError = -0.0056857983784047855 | LR = 0.1 | Epoch Train Time = 0.23Sec
$$ Epoch: 75 | Error = 0.106301 | DeltaError = -0.0021575441465718076 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 76 | Error = 0.108227 | DeltaError = 0.0019266298166682738 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 77 | Error = 0.107681 | DeltaError = -0.0005465674715711827 | LR = 0.1 | Epoch Train Time = 0.23Sec
$$ Epoch: 78 | Error = 0.108207 | DeltaError = 0.0005259358384364643 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch: 79 | Error = 0.102182 | DeltaError = -0.0060245668843481315 | LR = 0.1 | Epoch Train Time = 0.29Sec
$$ Epoch: 80 | Error = 0.103011 | DeltaError = 0.0008291204368074123 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 81 | Error = 0.102013 | DeltaError = -0.0009979524408240126 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 82 | Error = 0.101302 | DeltaError = -0.0007118389591451724 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 83 

$$ Epoch: 149 | Error = 0.076514 | DeltaError = 6.63310277445317e-05 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Epoch: 150 | Error = 0.077005 | DeltaError = 0.0004907105769514641 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 151 | Error = 0.073632 | DeltaError = -0.0033729115985933894 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch: 152 | Error = 0.074356 | DeltaError = 0.0007239651516557227 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 153 | Error = 0.077177 | DeltaError = 0.0028207795874161112 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 154 | Error = 0.0752 | DeltaError = -0.001976642474845622 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch: 155 | Error = 0.075043 | DeltaError = -0.00015759588595511131 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 156 | Error = 0.076913 | DeltaError = 0.0018709120978862986 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 157 | Error = 0.075814 | DeltaError = -0.0010999662391025372 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch:

$$ Epoch: 224 | Error = 0.059457 | DeltaError = -0.0007544141941006749 | LR = 0.1 | Epoch Train Time = 0.29Sec
$$ Epoch: 225 | Error = 0.061001 | DeltaError = 0.00154404304121096 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 226 | Error = 0.059222 | DeltaError = -0.0017788148845337623 | LR = 0.1 | Epoch Train Time = 0.29Sec
$$ Epoch: 227 | Error = 0.058749 | DeltaError = -0.00047336528807585126 | LR = 0.1 | Epoch Train Time = 0.25Sec
$$ Epoch: 228 | Error = 0.057461 | DeltaError = -0.0012881854732422066 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Epoch: 229 | Error = 0.057675 | DeltaError = 0.00021456122663973676 | LR = 0.1 | Epoch Train Time = 0.25Sec
$$ Epoch: 230 | Error = 0.058459 | DeltaError = 0.0007841014143509032 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch: 231 | Error = 0.056936 | DeltaError = -0.001523343253176712 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 232 | Error = 0.057357 | DeltaError = 0.00042137294768109646 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Ep

$$ Epoch: 299 | Error = 0.047811 | DeltaError = 0.0004198038622422948 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Epoch: 300 | Error = 0.048326 | DeltaError = 0.0005154636450935626 | LR = 0.1 | Epoch Train Time = 0.25Sec
$$ Epoch: 301 | Error = 0.049139 | DeltaError = 0.0008129288473859644 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Epoch: 302 | Error = 0.047064 | DeltaError = -0.0020748998755133474 | LR = 0.1 | Epoch Train Time = 0.25Sec
$$ Epoch: 303 | Error = 0.04791 | DeltaError = 0.0008455245639415673 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 304 | Error = 0.046839 | DeltaError = -0.0010706159108618898 | LR = 0.1 | Epoch Train Time = 0.29Sec
$$ Epoch: 305 | Error = 0.046195 | DeltaError = -0.0006440181630290032 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch: 306 | Error = 0.047082 | DeltaError = 0.0008864457738832002 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch: 307 | Error = 0.047364 | DeltaError = 0.00028241545199910595 | LR = 0.1 | Epoch Train Time = 0.34Sec
$$ Epoc

...

$$ Epoch: 749 | Error = 0.02187 | DeltaError = -6.77706584712584e-06 | LR = 0.1 | Epoch Train Time = 0.35Sec
$$ Epoch: 750 | Error = 0.021861 | DeltaError = -8.95770511511601e-06 | LR = 0.1 | Epoch Train Time = 0.35Sec
$$ Epoch: 751 | Error = 0.021851 | DeltaError = -9.3780287709587e-06 | LR = 0.1 | Epoch Train Time = 0.36Sec
$$ Epoch: 752 | Error = 0.021849 | DeltaError = -2.30028407082611e-06 | LR = 0.1 | Epoch Train Time = 0.35Sec
$$ Epoch: 753 | Error = 0.021836 | DeltaError = -1.297295542929816e-05 | LR = 0.1 | Epoch Train Time = 0.34Sec
$$ Epoch: 754 | Error = 0.02183 | DeltaError = -6.15065744247537e-06 | LR = 0.1 | Epoch Train Time = 0.36Sec
$$ Epoch: 755 | Error = 0.021822 | DeltaError = -8.07836883082516e-06 | LR = 0.1 | Epoch Train Time = 0.34Sec
$$ Epoch: 756 | Error = 0.021811 | DeltaError = -1.127714291321649e-05 | LR = 0.1 | Epoch Train Time = 0.35Sec
$$ Epoch: 757 | Error = 0.021805 | DeltaError = -5.92654857406449e-06 | LR = 0.1 | Epoch Train Time = 0.37Sec
$$ Epoch: 7

$$ Epoch: 824 | Error = 0.021298 | DeltaError = -8.34170141270074e-06 | LR = 0.1 | Epoch Train Time = 0.38Sec
$$ Epoch: 825 | Error = 0.021292 | DeltaError = -6.26886776974173e-06 | LR = 0.1 | Epoch Train Time = 0.38Sec
$$ Epoch: 826 | Error = 0.021284 | DeltaError = -7.05838142983986e-06 | LR = 0.1 | Epoch Train Time = 0.36Sec
$$ Epoch: 827 | Error = 0.021276 | DeltaError = -8.43902552746106e-06 | LR = 0.1 | Epoch Train Time = 0.37Sec
$$ Epoch: 828 | Error = 0.02127 | DeltaError = -6.19566541852634e-06 | LR = 0.1 | Epoch Train Time = 0.37Sec
$$ Epoch: 829 | Error = 0.021263 | DeltaError = -6.65276251719457e-06 | LR = 0.1 | Epoch Train Time = 0.37Sec
$$ Epoch: 830 | Error = 0.021255 | DeltaError = -8.00352978744603e-06 | LR = 0.1 | Epoch Train Time = 0.38Sec
$$ Epoch: 831 | Error = 0.02125 | DeltaError = -5.37672171596962e-06 | LR = 0.1 | Epoch Train Time = 0.36Sec
$$ Epoch: 832 | Error = 0.021241 | DeltaError = -8.58014109292662e-06 | LR = 0.1 | Epoch Train Time = 0.39Sec
$$ Epoch: 83

$$ Epoch: 899 | Error = 0.020815 | DeltaError = -4.85001032247967e-06 | LR = 0.1 | Epoch Train Time = 0.34Sec
$$ Epoch: 900 | Error = 0.020808 | DeltaError = -7.65246230901434e-06 | LR = 0.1 | Epoch Train Time = 0.36Sec
$$ Epoch: 901 | Error = 0.020804 | DeltaError = -3.96683714940296e-06 | LR = 0.1 | Epoch Train Time = 0.37Sec
$$ Epoch: 902 | Error = 0.020795 | DeltaError = -8.43713214111405e-06 | LR = 0.1 | Epoch Train Time = 0.35Sec
$$ Epoch: 903 | Error = 0.020793 | DeltaError = -2.24829958753864e-06 | LR = 0.1 | Epoch Train Time = 0.38Sec
$$ Epoch: 904 | Error = 0.020785 | DeltaError = -8.08550951952175e-06 | LR = 0.1 | Epoch Train Time = 0.34Sec
$$ Epoch: 905 | Error = 0.020778 | DeltaError = -7.29105323148821e-06 | LR = 0.1 | Epoch Train Time = 0.38Sec
$$ Epoch: 906 | Error = 0.020787 | DeltaError = 9.4731938238661e-06 | LR = 0.1 | Epoch Train Time = 0.34Sec
$$ Epoch: 907 | Error = 0.020766 | DeltaError = -2.12263085499452e-05 | LR = 0.1 | Epoch Train Time = 0.35Sec
$$ Epoch: 90

$$ Epoch: 974 | Error = 0.020378 | DeltaError = -5.83592962265234e-06 | LR = 0.1 | Epoch Train Time = 0.36Sec
$$ Epoch: 975 | Error = 0.020371 | DeltaError = -6.98245427234456e-06 | LR = 0.1 | Epoch Train Time = 0.35Sec
$$ Epoch: 976 | Error = 0.020367 | DeltaError = -3.81674455420672e-06 | LR = 0.1 | Epoch Train Time = 0.36Sec
$$ Epoch: 977 | Error = 0.020367 | DeltaError = -3.978058291873e-07 | LR = 0.1 | Epoch Train Time = 0.33Sec
$$ Epoch: 978 | Error = 0.020363 | DeltaError = -3.780049057895e-06 | LR = 0.1 | Epoch Train Time = 0.34Sec
$$ Epoch: 979 | Error = 0.020353 | DeltaError = -9.5155545496127e-06 | LR = 0.1 | Epoch Train Time = 0.36Sec
$$ Epoch: 980 | Error = 0.020347 | DeltaError = -5.88242434419045e-06 | LR = 0.1 | Epoch Train Time = 0.35Sec
$$ Epoch: 981 | Error = 0.020343 | DeltaError = -4.18534807640134e-06 | LR = 0.1 | Epoch Train Time = 0.36Sec
$$ Epoch: 982 | Error = 0.020337 | DeltaError = -6.56203205557856e-06 | LR = 0.1 | Epoch Train Time = 0.34Sec
$$ Epoch: 983 |

$$ Epoch: 1049 | Error = 0.02 | DeltaError = -3.05273073362283e-06 | LR = 0.1 | Epoch Train Time = 0.34Sec

%% Total Epochs =1049 | Epsilon = [0.02, 1e-06] | Total Learning Time = 7.54Min
13000 (13000,)
6500 (6500,)
Train Accuracy = 97.438%
Test Accuracy = 88.385%


In [139]:
nodesInHiddenLayers = [50]
print("@@@@@@@@@@---------Q1_b---------@@@@@@@@@@")
print("######------HiddenLayer = {}-----######\n".format(nodesInHiddenLayers))
neu_net = NeuralNetwork(nodesInHiddenLayers, "Sigmoid", "Sigmoid")
print(neu_net)
neu_net.fit(DTrain, oneHotTrainLables, eta=0.1, max_epoch=10000, epsilon=[0.02, 1e-06])

AccuTrain = neu_net.predict(DTrain, LabelsTrain)
AccuTest = neu_net.predict(DTest, LabelsTest)
print("Train Accuracy = {}%".format(round(AccuTrain,3)))
print("Test Accuracy = {}%".format(round(AccuTest,3)))

@@@@@@@@@@---------Q1_b---------@@@@@@@@@@
######------HiddenLayer = [50]-----######

Neural Network:
Layer 0: <Hidden Layer | Num_Perceptrons_Units = 50, Weights = (785, 50), Activation = Sigmoid>
Layer 1: <Output Layer | Num_Perceptrons_Units = 26, Weights = (50, 26), Activation = Sigmoid>

$$ Epoch: 1 | Error = 0.481675 | DeltaError = -2.695673670750729 | LR = 0.1 | Epoch Train Time = 0.23Sec
$$ Epoch: 2 | Error = 0.479711 | DeltaError = -0.0019647682574940317 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 3 | Error = 0.478959 | DeltaError = -0.0007512171866088657 | LR = 0.1 | Epoch Train Time = 0.22Sec
$$ Epoch: 4 | Error = 0.478201 | DeltaError = -0.0007579915734997611 | LR = 0.1 | Epoch Train Time = 0.21Sec
$$ Epoch: 5 | Error = 0.477286 | DeltaError = -0.0009150211183587831 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch: 6 | Error = 0.476128 | DeltaError = -0.0011584810304913207 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 7 | Error = 0.474697 | DeltaError = -0.0014303

$$ Epoch: 74 | Error = 0.175068 | DeltaError = -0.0008480535120994059 | LR = 0.1 | Epoch Train Time = 0.19Sec
$$ Epoch: 75 | Error = 0.176201 | DeltaError = 0.0011328623077383748 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 76 | Error = 0.176269 | DeltaError = 6.846825233097764e-05 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 77 | Error = 0.174043 | DeltaError = -0.0022256827865102524 | LR = 0.1 | Epoch Train Time = 0.23Sec
$$ Epoch: 78 | Error = 0.169949 | DeltaError = -0.004094869977606774 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 79 | Error = 0.1703 | DeltaError = 0.0003515468378547659 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 80 | Error = 0.173038 | DeltaError = 0.0027380743073708025 | LR = 0.1 | Epoch Train Time = 0.22Sec
$$ Epoch: 81 | Error = 0.167544 | DeltaError = -0.005494659645701977 | LR = 0.1 | Epoch Train Time = 0.19Sec
$$ Epoch: 82 | Error = 0.16399 | DeltaError = -0.003553109122464787 | LR = 0.1 | Epoch Train Time = 0.2Sec
$$ Epoch: 83 | Error 

$$ Epoch: 150 | Error = 0.126632 | DeltaError = 0.002604277660077439 | LR = 0.1 | Epoch Train Time = 0.19Sec
$$ Epoch: 151 | Error = 0.127154 | DeltaError = 0.0005220884283832772 | LR = 0.1 | Epoch Train Time = 0.21Sec
$$ Epoch: 152 | Error = 0.119297 | DeltaError = -0.007856700037559186 | LR = 0.1 | Epoch Train Time = 0.19Sec
$$ Epoch: 153 | Error = 0.12008 | DeltaError = 0.000782919384778924 | LR = 0.1 | Epoch Train Time = 0.19Sec
$$ Epoch: 154 | Error = 0.1218 | DeltaError = 0.0017193653969627407 | LR = 0.1 | Epoch Train Time = 0.2Sec
$$ Epoch: 155 | Error = 0.126297 | DeltaError = 0.004496808093570712 | LR = 0.1 | Epoch Train Time = 0.2Sec
$$ Epoch: 156 | Error = 0.120174 | DeltaError = -0.006122498727082107 | LR = 0.1 | Epoch Train Time = 0.21Sec
$$ Epoch: 157 | Error = 0.12357 | DeltaError = 0.00339548957379257 | LR = 0.1 | Epoch Train Time = 0.22Sec
$$ Epoch: 158 | Error = 0.12017 | DeltaError = -0.0033997630630484693 | LR = 0.1 | Epoch Train Time = 0.25Sec
$$ Epoch: 159 | Error

$$ Epoch: 225 | Error = 0.10676 | DeltaError = -7.754682493298082e-05 | LR = 0.1 | Epoch Train Time = 0.21Sec
$$ Epoch: 226 | Error = 0.103555 | DeltaError = -0.0032044466905673114 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 227 | Error = 0.102036 | DeltaError = -0.0015191279314723805 | LR = 0.1 | Epoch Train Time = 0.2Sec
$$ Epoch: 228 | Error = 0.105133 | DeltaError = 0.0030966306994871106 | LR = 0.1 | Epoch Train Time = 0.2Sec
$$ Epoch: 229 | Error = 0.104321 | DeltaError = -0.0008118331803311647 | LR = 0.1 | Epoch Train Time = 0.2Sec
$$ Epoch: 230 | Error = 0.105593 | DeltaError = 0.001272115988732822 | LR = 0.1 | Epoch Train Time = 0.25Sec
$$ Epoch: 231 | Error = 0.110894 | DeltaError = 0.0053010884366807565 | LR = 0.1 | Epoch Train Time = 0.25Sec
$$ Epoch: 232 | Error = 0.107392 | DeltaError = -0.0035022907918359496 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 233 | Error = 0.109485 | DeltaError = 0.002092620506342735 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Epoch: 2

$$ Epoch: 300 | Error = 0.093721 | DeltaError = -0.00017197844968230847 | LR = 0.1 | Epoch Train Time = 0.23Sec
$$ Epoch: 301 | Error = 0.090852 | DeltaError = -0.002868590219769762 | LR = 0.1 | Epoch Train Time = 0.25Sec
$$ Epoch: 302 | Error = 0.097132 | DeltaError = 0.006279480604798382 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 303 | Error = 0.092886 | DeltaError = -0.004246009233184869 | LR = 0.1 | Epoch Train Time = 0.23Sec
$$ Epoch: 304 | Error = 0.096873 | DeltaError = 0.00398732069871259 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 305 | Error = 0.094147 | DeltaError = -0.0027264595766447602 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 306 | Error = 0.094026 | DeltaError = -0.00012121449803935425 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 307 | Error = 0.09481 | DeltaError = 0.000784472184967494 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 308 | Error = 0.098116 | DeltaError = 0.003305814085397679 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 3

...

$$ Epoch: 3276 | Error = 0.028925 | DeltaError = -1.48433964643202e-06 | LR = 0.1 | Epoch Train Time = 0.3Sec
$$ Epoch: 3277 | Error = 0.028925 | DeltaError = 3.4317100998382e-07 | LR = 0.1 | Epoch Train Time = 0.25Sec
$$ Epoch: 3278 | Error = 0.028924 | DeltaError = -1.65316307262908e-06 | LR = 0.1 | Epoch Train Time = 0.25Sec
$$ Epoch: 3279 | Error = 0.02892 | DeltaError = -3.56489632851245e-06 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch: 3280 | Error = 0.02892 | DeltaError = -1.8359201652e-08 | LR = 0.1 | Epoch Train Time = 0.29Sec
$$ Epoch: 3281 | Error = 0.028918 | DeltaError = -2.48647836660273e-06 | LR = 0.1 | Epoch Train Time = 0.23Sec
$$ Epoch: 3282 | Error = 0.028917 | DeltaError = -8.4671775749112e-07 | LR = 0.1 | Epoch Train Time = 0.25Sec
$$ Epoch: 3283 | Error = 0.028915 | DeltaError = -2.30058328041724e-06 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Epoch: 3284 | Error = 0.028914 | DeltaError = -7.9172889224863e-07 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 32

$$ Epoch: 3351 | Error = 0.028822 | DeltaError = -4.1655648858746e-07 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 3352 | Error = 0.02882 | DeltaError = -1.9354844945918e-06 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 3353 | Error = 0.028819 | DeltaError = -6.0132892155257e-07 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch: 3354 | Error = 0.028817 | DeltaError = -2.4126334331083e-06 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Epoch: 3355 | Error = 0.028816 | DeltaError = -7.1198319969573e-07 | LR = 0.1 | Epoch Train Time = 0.25Sec
$$ Epoch: 3356 | Error = 0.028814 | DeltaError = -1.92216960899147e-06 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 3357 | Error = 0.028814 | DeltaError = -6.176894341188e-07 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 3358 | Error = 0.028813 | DeltaError = -1.04575357470077e-06 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 3359 | Error = 0.02881 | DeltaError = -2.32806974732186e-06 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Epoch: 

$$ Epoch: 3426 | Error = 0.028721 | DeltaError = -2.66206424618873e-06 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Epoch: 3427 | Error = 0.02872 | DeltaError = -1.31027444509255e-06 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 3428 | Error = 0.028719 | DeltaError = -6.3541619394272e-07 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 3429 | Error = 0.028717 | DeltaError = -1.65755972929282e-06 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 3430 | Error = 0.028716 | DeltaError = -1.17241611564262e-06 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 3431 | Error = 0.028715 | DeltaError = -1.26896011801852e-06 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 3432 | Error = 0.028714 | DeltaError = -9.3648462231102e-07 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 3433 | Error = 0.028712 | DeltaError = -2.07479670261135e-06 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 3434 | Error = 0.02871 | DeltaError = -1.98094905346641e-06 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Ep

$$ Epoch: 3501 | Error = 0.028626 | DeltaError = -2.18246011165352e-06 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch: 3502 | Error = 0.028625 | DeltaError = -6.9604181372057e-07 | LR = 0.1 | Epoch Train Time = 0.25Sec
$$ Epoch: 3503 | Error = 0.028625 | DeltaError = -3.5114020421889e-07 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 3504 | Error = 0.028622 | DeltaError = -2.17658635737195e-06 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch: 3505 | Error = 0.02862 | DeltaError = -2.19775261136104e-06 | LR = 0.1 | Epoch Train Time = 0.25Sec
$$ Epoch: 3506 | Error = 0.028619 | DeltaError = -1.23854341098004e-06 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Epoch: 3507 | Error = 0.028618 | DeltaError = -8.8717025638341e-07 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch: 3508 | Error = 0.028616 | DeltaError = -1.78808819479431e-06 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 3509 | Error = 0.028615 | DeltaError = -1.53155883435285e-06 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Ep

$$ Epoch: 3576 | Error = 0.028533 | DeltaError = -9.9946843678742e-07 | LR = 0.1 | Epoch Train Time = 0.3Sec
$$ Epoch: 3577 | Error = 0.02853 | DeltaError = -2.26282110030521e-06 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 3578 | Error = 0.02853 | DeltaError = -6.3767722881594e-07 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 3579 | Error = 0.028529 | DeltaError = -1.05380749159181e-06 | LR = 0.1 | Epoch Train Time = 0.28Sec
$$ Epoch: 3580 | Error = 0.028527 | DeltaError = -1.74082488093794e-06 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch: 3581 | Error = 0.028526 | DeltaError = -9.9537060439803e-07 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 3582 | Error = 0.028525 | DeltaError = -8.3375460620261e-07 | LR = 0.1 | Epoch Train Time = 0.27Sec
$$ Epoch: 3583 | Error = 0.028524 | DeltaError = -1.44506850166537e-06 | LR = 0.1 | Epoch Train Time = 0.25Sec
$$ Epoch: 3584 | Error = 0.028523 | DeltaError = -9.3333368227783e-07 | LR = 0.1 | Epoch Train Time = 0.26Sec
$$ Epoch:

In [140]:
'''Global Params'''
nodesInHiddenLayers = [10]
print("@@@@@@@@@@---------Q1_b---------@@@@@@@@@@")
print("######------HiddenLayer = {}-----######\n".format(nodesInHiddenLayers))
neu_net = NeuralNetwork(nodesInHiddenLayers, "Sigmoid", "Sigmoid")
print(neu_net)
neu_net.fit(DTrain, oneHotTrainLables, eta=0.1, max_epoch=10000, epsilon=[0.02, 1e-06])

AccuTrain = neu_net.predict(DTrain, LabelsTrain)
AccuTest = neu_net.predict(DTest, LabelsTest)
print("Train Accuracy = {}%".format(round(AccuTrain,3)))
print("Test Accuracy = {}%".format(round(AccuTest,3)))

@@@@@@@@@@---------Q1_b---------@@@@@@@@@@
######------HiddenLayer = [10]-----######

Neural Network:
Layer 0: <Hidden Layer | Num_Perceptrons_Units = 10, Weights = (785, 10), Activation = Sigmoid>
Layer 1: <Output Layer | Num_Perceptrons_Units = 26, Weights = (10, 26), Activation = Sigmoid>

$$ Epoch: 1 | Error = 0.516951 | DeltaError = -2.686352394668197 | LR = 0.1 | Epoch Train Time = 0.25Sec
$$ Epoch: 2 | Error = 0.491714 | DeltaError = -0.025236779432826373 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch: 3 | Error = 0.485696 | DeltaError = -0.006018682258476127 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 4 | Error = 0.48338 | DeltaError = -0.0023154743983302972 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 5 | Error = 0.48228 | DeltaError = -0.0010997646537651007 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch: 6 | Error = 0.481695 | DeltaError = -0.0005856972415284045 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch: 7 | Error = 0.481356 | DeltaError = -0.00033861251

$$ Epoch: 74 | Error = 0.475769 | DeltaError = -8.631091893562814e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 75 | Error = 0.475666 | DeltaError = -0.00010373934663648889 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 76 | Error = 0.475639 | DeltaError = -2.633873757768024e-05 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch: 77 | Error = 0.475571 | DeltaError = -6.815019068734385e-05 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 78 | Error = 0.475397 | DeltaError = -0.0001739230677706427 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 79 | Error = 0.475328 | DeltaError = -6.882986916512124e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 80 | Error = 0.47521 | DeltaError = -0.00011830820951419518 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch: 81 | Error = 0.475164 | DeltaError = -4.598022853513628e-05 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch: 82 | Error = 0.475024 | DeltaError = -0.0001403514410104334 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch:

$$ Epoch: 150 | Error = 0.470938 | DeltaError = -4.177263025967326e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 151 | Error = 0.470841 | DeltaError = -9.723780266207573e-05 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch: 152 | Error = 0.470766 | DeltaError = -7.478128421550823e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 153 | Error = 0.470806 | DeltaError = 4.026617808911004e-05 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 154 | Error = 0.470686 | DeltaError = -0.00012032777492115843 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 155 | Error = 0.47058 | DeltaError = -0.00010613503741463413 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 156 | Error = 0.470608 | DeltaError = 2.761386318550496e-05 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 157 | Error = 0.470557 | DeltaError = -5.091130549794709e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 158 | Error = 0.470569 | DeltaError = 1.203516285480655e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ 

$$ Epoch: 225 | Error = 0.468945 | DeltaError = -0.0001107587213114436 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch: 226 | Error = 0.468936 | DeltaError = -8.54925435272991e-06 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 227 | Error = 0.46888 | DeltaError = -5.61192897286511e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 228 | Error = 0.4689 | DeltaError = 2.004423966039948e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 229 | Error = 0.469136 | DeltaError = 0.00023610790089545475 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 230 | Error = 0.468948 | DeltaError = -0.00018847402161659144 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 231 | Error = 0.4689 | DeltaError = -4.786546931673241e-05 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 232 | Error = 0.469307 | DeltaError = 0.0004065771019458598 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 233 | Error = 0.468744 | DeltaError = -0.0005627622938547683 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch:

$$ Epoch: 300 | Error = 0.467849 | DeltaError = -0.00018613069253087255 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 301 | Error = 0.468075 | DeltaError = 0.0002262093413089028 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 302 | Error = 0.46783 | DeltaError = -0.0002445022514835915 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 303 | Error = 0.46836 | DeltaError = 0.000529320169231795 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 304 | Error = 0.467888 | DeltaError = -0.0004719430908740052 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 305 | Error = 0.467836 | DeltaError = -5.184584175083051e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 306 | Error = 0.467887 | DeltaError = 5.065011912414041e-05 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 307 | Error = 0.46782 | DeltaError = -6.671142434883093e-05 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch: 308 | Error = 0.46803 | DeltaError = 0.00021035429613852627 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch

...

$$ Epoch: 2593 | Error = 0.465018 | DeltaError = -4.915104798197678e-05 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch: 2594 | Error = 0.46506 | DeltaError = 4.113514918302474e-05 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 2595 | Error = 0.465377 | DeltaError = 0.00031779045516350246 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 2596 | Error = 0.465039 | DeltaError = -0.0003381325143866909 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 2597 | Error = 0.465048 | DeltaError = 8.63115994276242e-06 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 2598 | Error = 0.465024 | DeltaError = -2.396432430712414e-05 | LR = 0.1 | Epoch Train Time = 0.19Sec
$$ Epoch: 2599 | Error = 0.465081 | DeltaError = 5.735830292341593e-05 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 2600 | Error = 0.465025 | DeltaError = -5.614405480736107e-05 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 2601 | Error = 0.46505 | DeltaError = 2.529111499582904e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec

$$ Epoch: 2739 | Error = 0.465075 | DeltaError = 4.757385915776791e-05 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 2740 | Error = 0.465056 | DeltaError = -1.902492076183426e-05 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 2741 | Error = 0.465032 | DeltaError = -2.457363619118302e-05 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 2742 | Error = 0.464976 | DeltaError = -5.551025049144176e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 2743 | Error = 0.464992 | DeltaError = 1.556309923683718e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 2744 | Error = 0.465007 | DeltaError = 1.528429331820735e-05 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 2745 | Error = 0.464981 | DeltaError = -2.59221017845368e-05 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch: 2746 | Error = 0.464963 | DeltaError = -1.871226793276337e-05 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 2747 | Error = 0.465033 | DeltaError = 7.081880418641218e-05 | LR = 0.1 | Epoch Train Time = 0.16S

$$ Epoch: 2886 | Error = 0.464957 | DeltaError = -4.230317365921943e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 2887 | Error = 0.46501 | DeltaError = 5.252243845027538e-05 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 2888 | Error = 0.465032 | DeltaError = 2.226073406130791e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 2889 | Error = 0.464951 | DeltaError = -8.11651139925873e-05 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 2890 | Error = 0.46495 | DeltaError = -1.16817770146049e-06 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 2891 | Error = 0.464944 | DeltaError = -5.46163920650544e-06 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 2892 | Error = 0.465053 | DeltaError = 0.00010899951410675568 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 2893 | Error = 0.465015 | DeltaError = -3.872073582522617e-05 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 2894 | Error = 0.464964 | DeltaError = -5.109626569593795e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec

$$ Epoch: 2960 | Error = 0.464962 | DeltaError = 1.220607003049601e-05 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 2961 | Error = 0.464938 | DeltaError = -2.44702456270729e-05 | LR = 0.1 | Epoch Train Time = 0.18Sec
$$ Epoch: 2962 | Error = 0.464938 | DeltaError = 1.7121680045973e-07 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch: 2963 | Error = 0.464973 | DeltaError = 3.464073297421155e-05 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch: 2964 | Error = 0.464959 | DeltaError = -1.387865688123879e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 2965 | Error = 0.464949 | DeltaError = -9.8726947787009e-06 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 2966 | Error = 0.465031 | DeltaError = 8.229560714495543e-05 | LR = 0.1 | Epoch Train Time = 0.14Sec
$$ Epoch: 2967 | Error = 0.464987 | DeltaError = -4.46372087943292e-05 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 2968 | Error = 0.464999 | DeltaError = 1.215362830930822e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ 

$$ Epoch: 3034 | Error = 0.465 | DeltaError = 4.434804898068023e-05 | LR = 0.1 | Epoch Train Time = 0.14Sec
$$ Epoch: 3035 | Error = 0.465269 | DeltaError = 0.00026862009152073263 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 3036 | Error = 0.465097 | DeltaError = -0.00017214846277679996 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 3037 | Error = 0.464962 | DeltaError = -0.0001341601169861928 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 3038 | Error = 0.464938 | DeltaError = -2.41136407609921e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 3039 | Error = 0.464959 | DeltaError = 2.043274363860981e-05 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 3040 | Error = 0.464936 | DeltaError = -2.319744820183933e-05 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 3041 | Error = 0.464974 | DeltaError = 3.874333358722115e-05 | LR = 0.1 | Epoch Train Time = 0.17Sec
$$ Epoch: 3042 | Error = 0.465049 | DeltaError = 7.453497953757715e-05 | LR = 0.1 | Epoch Train Time = 0.15Sec

In [141]:
'''Global Params'''
nodesInHiddenLayers = [5]
print("@@@@@@@@@@---------Q1_b---------@@@@@@@@@@")
print("######------HiddenLayer = {}-----######\n".format(nodesInHiddenLayers))
neu_net = NeuralNetwork(nodesInHiddenLayers, "Sigmoid", "Sigmoid")
print(neu_net)
neu_net.fit(DTrain, oneHotTrainLables, eta=0.1, max_epoch=10000, epsilon=[0.02, 1e-06])

AccuTrain = neu_net.predict(DTrain, LabelsTrain)
AccuTest = neu_net.predict(DTest, LabelsTest)
print("Train Accuracy = {}%".format(round(AccuTrain,3)))
print("Test Accuracy = {}%".format(round(AccuTest,3)))

@@@@@@@@@@---------Q1_b---------@@@@@@@@@@
######------HiddenLayer = [5]-----######

Neural Network:
Layer 0: <Hidden Layer | Num_Perceptrons_Units = 5, Weights = (785, 5), Activation = Sigmoid>
Layer 1: <Output Layer | Num_Perceptrons_Units = 26, Weights = (5, 26), Activation = Sigmoid>

$$ Epoch: 1 | Error = 0.579076 | DeltaError = -2.6598766947543346 | LR = 0.1 | Epoch Train Time = 0.24Sec
$$ Epoch: 2 | Error = 0.5128 | DeltaError = -0.06627601940156969 | LR = 0.1 | Epoch Train Time = 0.14Sec
$$ Epoch: 3 | Error = 0.496589 | DeltaError = -0.016210892331085802 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 4 | Error = 0.490027 | DeltaError = -0.006561605004694215 | LR = 0.1 | Epoch Train Time = 0.13Sec
$$ Epoch: 5 | Error = 0.486708 | DeltaError = -0.0033187202477741318 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 6 | Error = 0.484807 | DeltaError = -0.0019014907508856371 | LR = 0.1 | Epoch Train Time = 0.16Sec
$$ Epoch: 7 | Error = 0.483626 | DeltaError = -0.0011809549080241

In [142]:
'''Global Params'''
nodesInHiddenLayers = [1]
print("@@@@@@@@@@---------Q1_b---------@@@@@@@@@@")
print("######------HiddenLayer = {}-----######\n".format(nodesInHiddenLayers))
neu_net = NeuralNetwork(nodesInHiddenLayers, "Sigmoid", "Sigmoid")
print(neu_net)
neu_net.fit(DTrain, oneHotTrainLables, eta=0.1, max_epoch=10000, epsilon=[0.02, 1e-06])

AccuTrain = neu_net.predict(DTrain, LabelsTrain)
AccuTest = neu_net.predict(DTest, LabelsTest)
print("Train Accuracy = {}%".format(round(AccuTrain,3)))
print("Test Accuracy = {}%".format(round(AccuTest,3)))

@@@@@@@@@@---------Q1_b---------@@@@@@@@@@
######------HiddenLayer = [1]-----######

Neural Network:
Layer 0: <Hidden Layer | Num_Perceptrons_Units = 1, Weights = (785, 1), Activation = Sigmoid>
Layer 1: <Output Layer | Num_Perceptrons_Units = 26, Weights = (1, 26), Activation = Sigmoid>

$$ Epoch: 1 | Error = 1.217765 | DeltaError = -2.054817667180516 | LR = 0.1 | Epoch Train Time = 0.23Sec
$$ Epoch: 2 | Error = 0.805329 | DeltaError = -0.4124356767195626 | LR = 0.1 | Epoch Train Time = 0.12Sec
$$ Epoch: 3 | Error = 0.668654 | DeltaError = -0.13667526990352052 | LR = 0.1 | Epoch Train Time = 0.14Sec
$$ Epoch: 4 | Error = 0.605654 | DeltaError = -0.06299985562605792 | LR = 0.1 | Epoch Train Time = 0.14Sec
$$ Epoch: 5 | Error = 0.570806 | DeltaError = -0.03484870846737664 | LR = 0.1 | Epoch Train Time = 0.13Sec
$$ Epoch: 6 | Error = 0.549219 | DeltaError = -0.02158661529137451 | LR = 0.1 | Epoch Train Time = 0.15Sec
$$ Epoch: 7 | Error = 0.534783 | DeltaError = -0.014435572226279557 | L

$$ Epoch: 74 | Error = 0.481124 | DeltaError = -1.651083344006432e-05 | LR = 0.1 | Epoch Train Time = 0.14Sec
$$ Epoch: 75 | Error = 0.481108 | DeltaError = -1.569666503509204e-05 | LR = 0.1 | Epoch Train Time = 0.14Sec
$$ Epoch: 76 | Error = 0.481093 | DeltaError = -1.492795326640817e-05 | LR = 0.1 | Epoch Train Time = 0.13Sec
$$ Epoch: 77 | Error = 0.481079 | DeltaError = -1.420195661339108e-05 | LR = 0.1 | Epoch Train Time = 0.14Sec
$$ Epoch: 78 | Error = 0.481066 | DeltaError = -1.351577553010408e-05 | LR = 0.1 | Epoch Train Time = 0.13Sec
$$ Epoch: 79 | Error = 0.481053 | DeltaError = -1.286696298596901e-05 | LR = 0.1 | Epoch Train Time = 0.14Sec
$$ Epoch: 80 | Error = 0.48104 | DeltaError = -1.225326371562252e-05 | LR = 0.1 | Epoch Train Time = 0.13Sec
$$ Epoch: 81 | Error = 0.481029 | DeltaError = -1.167237157301271e-05 | LR = 0.1 | Epoch Train Time = 0.11Sec
$$ Epoch: 82 | Error = 0.481018 | DeltaError = -1.112246055989807e-05 | LR = 0.1 | Epoch Train Time = 0.13Sec
$$ Epoch: 8

In [1]:
'''Global Params'''
nodesInHiddenLayers = [100]
print("@@@@@@@@@@---------Q1_c---------@@@@@@@@@@")
print("@@@@------Adaptive Learning Rate------@@@@")
print("######------HiddenLayer = {}-----######\n".format(nodesInHiddenLayers))
neu_net = NeuralNetwork(nodesInHiddenLayers, "Sigmoid", "Sigmoid")
print(neu_net)
neu_net.fit(DTrain, oneHotTrainLables, eta=0.5, max_epoch=10000, epsilon=[0.02, 1e-07], adaptive=True)

AccuTrain = neu_net.predict(DTrain, LabelsTrain)
AccuTest = neu_net.predict(DTest, LabelsTest)
print("Train Accuracy = {}%".format(round(AccuTrain,3)))
print("Test Accuracy = {}%".format(round(AccuTest,3)))

@@@@@@@@@@---------Q1_c---------@@@@@@@@@@
@@@@------Adaptive Learning Rate------@@@@
######------HiddenLayer = [100]-----######



NameError: name 'NeuralNetwork' is not defined

In [148]:
'''Global Params'''
nodesInHiddenLayers = [50]
print("@@@@@@@@@@---------Q1_c---------@@@@@@@@@@")
print("@@@@------Adaptive Learning Rate------@@@@")
print("######------HiddenLayer = {}-----######\n".format(nodesInHiddenLayers))
neu_net = NeuralNetwork(nodesInHiddenLayers, "Sigmoid", "Sigmoid")
print(neu_net)
neu_net.fit(DTrain, oneHotTrainLables, eta=0.5, max_epoch=20000, epsilon=[0.02, 1e-07], adaptive=True)

AccuTrain = neu_net.predict(DTrain, LabelsTrain)
AccuTest = neu_net.predict(DTest, LabelsTest)
print("Train Accuracy = {}%".format(round(AccuTrain,3)))
print("Test Accuracy = {}%".format(round(AccuTest,3)))

@@@@@@@@@@---------Q1_c---------@@@@@@@@@@
@@@@------Adaptive Learning Rate------@@@@
######------HiddenLayer = [50]-----######

Neural Network:
Layer 0: <Hidden Layer | Num_Perceptrons_Units = 50, Weights = (785, 50), Activation = Sigmoid>
Layer 1: <Output Layer | Num_Perceptrons_Units = 26, Weights = (50, 26), Activation = Sigmoid>

$$ Epoch: 1 | Error = 0.477673 | DeltaError = -2.69967586123818 | LR = 0.5 | Epoch Train Time = 0.23Sec
$$ Epoch: 2 | Error = 0.472053 | DeltaError = -0.005620406959382429 | LR = 0.35355 | Epoch Train Time = 0.15Sec
$$ Epoch: 3 | Error = 0.463165 | DeltaError = -0.008887218363369087 | LR = 0.28868 | Epoch Train Time = 0.17Sec
$$ Epoch: 4 | Error = 0.450567 | DeltaError = -0.012598777323054087 | LR = 0.25 | Epoch Train Time = 0.17Sec
$$ Epoch: 5 | Error = 0.436975 | DeltaError = -0.013592091734573264 | LR = 0.22361 | Epoch Train Time = 0.16Sec
$$ Epoch: 6 | Error = 0.423241 | DeltaError = -0.013733193249511222 | LR = 0.20412 | Epoch Train Time = 0.17Sec
$$

$$ Epoch: 71 | Error = 0.167662 | DeltaError = 0.0003859627304249047 | LR = 0.05934 | Epoch Train Time = 0.24Sec
$$ Epoch: 72 | Error = 0.163955 | DeltaError = -0.0037063307066212132 | LR = 0.05893 | Epoch Train Time = 0.21Sec
$$ Epoch: 73 | Error = 0.163351 | DeltaError = -0.000604349295911133 | LR = 0.05852 | Epoch Train Time = 0.23Sec
$$ Epoch: 74 | Error = 0.162561 | DeltaError = -0.0007894387804425195 | LR = 0.05812 | Epoch Train Time = 0.24Sec
$$ Epoch: 75 | Error = 0.165561 | DeltaError = 0.0029994244470664395 | LR = 0.05774 | Epoch Train Time = 0.21Sec
$$ Epoch: 76 | Error = 0.162745 | DeltaError = -0.0028159746609991154 | LR = 0.05735 | Epoch Train Time = 0.23Sec
$$ Epoch: 77 | Error = 0.164684 | DeltaError = 0.001939129727874689 | LR = 0.05698 | Epoch Train Time = 0.25Sec
$$ Epoch: 78 | Error = 0.15998 | DeltaError = -0.00470397486471244 | LR = 0.05661 | Epoch Train Time = 0.22Sec
$$ Epoch: 79 | Error = 0.163534 | DeltaError = 0.003554134928363817 | LR = 0.05625 | Epoch Train

$$ Epoch: 144 | Error = 0.125348 | DeltaError = 0.0010867869389362267 | LR = 0.04167 | Epoch Train Time = 0.25Sec
$$ Epoch: 145 | Error = 0.123812 | DeltaError = -0.0015365498780231107 | LR = 0.04152 | Epoch Train Time = 0.26Sec
$$ Epoch: 146 | Error = 0.122666 | DeltaError = -0.001146061781391608 | LR = 0.04138 | Epoch Train Time = 0.24Sec
$$ Epoch: 147 | Error = 0.12466 | DeltaError = 0.0019946746129916454 | LR = 0.04124 | Epoch Train Time = 0.24Sec
$$ Epoch: 148 | Error = 0.125443 | DeltaError = 0.0007821420113068162 | LR = 0.0411 | Epoch Train Time = 0.34Sec
$$ Epoch: 149 | Error = 0.123631 | DeltaError = -0.0018114947283637273 | LR = 0.04096 | Epoch Train Time = 0.3Sec
$$ Epoch: 150 | Error = 0.125122 | DeltaError = 0.001491029156331175 | LR = 0.04082 | Epoch Train Time = 0.23Sec
$$ Epoch: 151 | Error = 0.124404 | DeltaError = -0.0007180482448104625 | LR = 0.04069 | Epoch Train Time = 0.25Sec
$$ Epoch: 152 | Error = 0.123812 | DeltaError = -0.0005922938623848378 | LR = 0.04056 | E

$$ Epoch: 216 | Error = 0.10507 | DeltaError = -0.0012373439301103795 | LR = 0.03402 | Epoch Train Time = 0.23Sec
$$ Epoch: 217 | Error = 0.10416 | DeltaError = -0.0009096797608463824 | LR = 0.03394 | Epoch Train Time = 0.24Sec
$$ Epoch: 218 | Error = 0.105808 | DeltaError = 0.001647511741768054 | LR = 0.03386 | Epoch Train Time = 0.24Sec
$$ Epoch: 219 | Error = 0.10527 | DeltaError = -0.0005374566865731512 | LR = 0.03379 | Epoch Train Time = 0.25Sec
$$ Epoch: 220 | Error = 0.10439 | DeltaError = -0.0008801709859646706 | LR = 0.03371 | Epoch Train Time = 0.23Sec
$$ Epoch: 221 | Error = 0.107611 | DeltaError = 0.003221219059687863 | LR = 0.03363 | Epoch Train Time = 0.27Sec
$$ Epoch: 222 | Error = 0.104932 | DeltaError = -0.0026792457380707124 | LR = 0.03356 | Epoch Train Time = 0.23Sec
$$ Epoch: 223 | Error = 0.105669 | DeltaError = 0.0007364799823275648 | LR = 0.03348 | Epoch Train Time = 0.24Sec
$$ Epoch: 224 | Error = 0.104154 | DeltaError = -0.001514696321027728 | LR = 0.03341 | Ep

$$ Epoch: 288 | Error = 0.091787 | DeltaError = -0.0011979359537566264 | LR = 0.02946 | Epoch Train Time = 0.22Sec
$$ Epoch: 289 | Error = 0.090737 | DeltaError = -0.001050557282292655 | LR = 0.02941 | Epoch Train Time = 0.31Sec
$$ Epoch: 290 | Error = 0.091617 | DeltaError = 0.0008804818962976635 | LR = 0.02936 | Epoch Train Time = 0.25Sec
$$ Epoch: 291 | Error = 0.091369 | DeltaError = -0.00024774326161444704 | LR = 0.02931 | Epoch Train Time = 0.25Sec
$$ Epoch: 292 | Error = 0.091022 | DeltaError = -0.00034774286795080656 | LR = 0.02926 | Epoch Train Time = 0.25Sec
$$ Epoch: 293 | Error = 0.091704 | DeltaError = 0.0006822404260748982 | LR = 0.02921 | Epoch Train Time = 0.21Sec
$$ Epoch: 294 | Error = 0.090132 | DeltaError = -0.0015720188143336783 | LR = 0.02916 | Epoch Train Time = 0.23Sec
$$ Epoch: 295 | Error = 0.093179 | DeltaError = 0.0030473965341971804 | LR = 0.02911 | Epoch Train Time = 0.24Sec
$$ Epoch: 296 | Error = 0.09113 | DeltaError = -0.00204960732584826 | LR = 0.02906

...

$$ Epoch: 2592 | Error = 0.064691 | DeltaError = -4.24575282600981e-06 | LR = 0.00982 | Epoch Train Time = 0.23Sec
$$ Epoch: 2593 | Error = 0.064696 | DeltaError = 4.52380761563342e-06 | LR = 0.00982 | Epoch Train Time = 0.27Sec
$$ Epoch: 2594 | Error = 0.064688 | DeltaError = -7.9152720223602e-06 | LR = 0.00982 | Epoch Train Time = 0.24Sec
$$ Epoch: 2595 | Error = 0.064688 | DeltaError = 4.3969786915621e-07 | LR = 0.00982 | Epoch Train Time = 0.22Sec
$$ Epoch: 2596 | Error = 0.064689 | DeltaError = 6.7863620266395e-07 | LR = 0.00981 | Epoch Train Time = 0.26Sec
$$ Epoch: 2597 | Error = 0.064692 | DeltaError = 2.57368253889223e-06 | LR = 0.00981 | Epoch Train Time = 0.24Sec
$$ Epoch: 2598 | Error = 0.06468 | DeltaError = -1.171981388489729e-05 | LR = 0.00981 | Epoch Train Time = 0.27Sec
$$ Epoch: 2599 | Error = 0.064683 | DeltaError = 2.5663287191724e-06 | LR = 0.00981 | Epoch Train Time = 0.25Sec
$$ Epoch: 2600 | Error = 0.064674 | DeltaError = -8.47792018621174e-06 | LR = 0.00981 | E

$$ Epoch: 2664 | Error = 0.06454 | DeltaError = -6.68703066988852e-06 | LR = 0.00969 | Epoch Train Time = 0.34Sec
$$ Epoch: 2665 | Error = 0.064543 | DeltaError = 2.75723351483848e-06 | LR = 0.00969 | Epoch Train Time = 0.26Sec
$$ Epoch: 2666 | Error = 0.06454 | DeltaError = -2.52837345594836e-06 | LR = 0.00968 | Epoch Train Time = 0.23Sec
$$ Epoch: 2667 | Error = 0.064536 | DeltaError = -4.32322897013504e-06 | LR = 0.00968 | Epoch Train Time = 0.22Sec
$$ Epoch: 2668 | Error = 0.064537 | DeltaError = 8.5802987440098e-07 | LR = 0.00968 | Epoch Train Time = 0.22Sec
$$ Epoch: 2669 | Error = 0.06454 | DeltaError = 3.13543781003955e-06 | LR = 0.00968 | Epoch Train Time = 0.24Sec
$$ Epoch: 2670 | Error = 0.064533 | DeltaError = -6.46179929873758e-06 | LR = 0.00968 | Epoch Train Time = 0.23Sec
$$ Epoch: 2671 | Error = 0.06453 | DeltaError = -3.40723896014028e-06 | LR = 0.00967 | Epoch Train Time = 0.22Sec
$$ Epoch: 2672 | Error = 0.06453 | DeltaError = 3.1667322013262e-07 | LR = 0.00967 | Epo

$$ Epoch: 2736 | Error = 0.064398 | DeltaError = -5.79071652709318e-06 | LR = 0.00956 | Epoch Train Time = 0.24Sec
$$ Epoch: 2737 | Error = 0.064398 | DeltaError = -7.246187604115e-08 | LR = 0.00956 | Epoch Train Time = 0.22Sec
$$ Epoch: 2738 | Error = 0.064396 | DeltaError = -1.55883354019393e-06 | LR = 0.00956 | Epoch Train Time = 0.21Sec
$$ Epoch: 2739 | Error = 0.064393 | DeltaError = -3.44862958075176e-06 | LR = 0.00955 | Epoch Train Time = 0.22Sec
$$ Epoch: 2740 | Error = 0.064394 | DeltaError = 9.698253585072e-07 | LR = 0.00955 | Epoch Train Time = 0.23Sec
$$ Epoch: 2741 | Error = 0.06439 | DeltaError = -3.61893252384671e-06 | LR = 0.00955 | Epoch Train Time = 0.22Sec
$$ Epoch: 2742 | Error = 0.064391 | DeltaError = 6.2491411058352e-07 | LR = 0.00955 | Epoch Train Time = 0.23Sec
$$ Epoch: 2743 | Error = 0.064386 | DeltaError = -5.25100974292647e-06 | LR = 0.00955 | Epoch Train Time = 0.23Sec
$$ Epoch: 2744 | Error = 0.064384 | DeltaError = -1.86503636480373e-06 | LR = 0.00955 | 

$$ Epoch: 2808 | Error = 0.064259 | DeltaError = -8.8404157758126e-07 | LR = 0.00944 | Epoch Train Time = 0.23Sec
$$ Epoch: 2809 | Error = 0.064257 | DeltaError = -1.97128003240343e-06 | LR = 0.00943 | Epoch Train Time = 0.22Sec
$$ Epoch: 2810 | Error = 0.064259 | DeltaError = 1.80795117180055e-06 | LR = 0.00943 | Epoch Train Time = 0.22Sec
$$ Epoch: 2811 | Error = 0.064255 | DeltaError = -3.91402713759148e-06 | LR = 0.00943 | Epoch Train Time = 0.24Sec
$$ Epoch: 2812 | Error = 0.064259 | DeltaError = 4.14171646404005e-06 | LR = 0.00943 | Epoch Train Time = 0.21Sec
$$ Epoch: 2813 | Error = 0.064251 | DeltaError = -7.64674470056514e-06 | LR = 0.00943 | Epoch Train Time = 0.24Sec
$$ Epoch: 2814 | Error = 0.064248 | DeltaError = -3.32347251563725e-06 | LR = 0.00943 | Epoch Train Time = 0.21Sec
$$ Epoch: 2815 | Error = 0.06426 | DeltaError = 1.174352638803888e-05 | LR = 0.00942 | Epoch Train Time = 0.24Sec
$$ Epoch: 2816 | Error = 0.064244 | DeltaError = -1.609968060810318e-05 | LR = 0.009

KeyboardInterrupt: 

In [ ]:
'''Global Params'''
nodesInHiddenLayers = [10]
print("@@@@@@@@@@---------Q1_c---------@@@@@@@@@@")
print("@@@@------Adaptive Learning Rate------@@@@")
print("######------HiddenLayer = {}-----######\n".format(nodesInHiddenLayers))
neu_net = NeuralNetwork(nodesInHiddenLayers, "Sigmoid", "Sigmoid")
print(neu_net)
neu_net.fit(DTrain, oneHotTrainLables, eta=0.5, max_epoch=20000, epsilon=[0.02, 1e-07], adaptive=True)

AccuTrain = neu_net.predict(DTrain, LabelsTrain)
AccuTest = neu_net.predict(DTest, LabelsTest)
print("Train Accuracy = {}%".format(round(AccuTrain,3)))
print("Test Accuracy = {}%".format(round(AccuTest,3)))

In [ ]:
'''Global Params'''
nodesInHiddenLayers = [5]
print("@@@@@@@@@@---------Q1_c---------@@@@@@@@@@")
print("@@@@------Adaptive Learning Rate------@@@@")
print("######------HiddenLayer = {}-----######\n".format(nodesInHiddenLayers))
neu_net = NeuralNetwork(nodesInHiddenLayers, "Sigmoid", "Sigmoid")
print(neu_net)
neu_net.fit(DTrain, oneHotTrainLables, eta=0.5, max_epoch=20000, epsilon=[0.02, 1e-07], adaptive=True)

AccuTrain = neu_net.predict(DTrain, LabelsTrain)
AccuTest = neu_net.predict(DTest, LabelsTest)
print("Train Accuracy = {}%".format(round(AccuTrain,3)))
print("Test Accuracy = {}%".format(round(AccuTest,3)))

In [ ]:
'''Global Params'''
nodesInHiddenLayers = [1]
print("@@@@@@@@@@---------Q1_c---------@@@@@@@@@@")
print("@@@@------Adaptive Learning Rate------@@@@")
print("######------HiddenLayer = {}-----######\n".format(nodesInHiddenLayers))
neu_net = NeuralNetwork(nodesInHiddenLayers, "Sigmoid", "Sigmoid")
print(neu_net)
neu_net.fit(DTrain, oneHotTrainLables, eta=0.5, max_epoch=20000, epsilon=[0.02, 1e-07], adaptive=True)

AccuTrain = neu_net.predict(DTrain, LabelsTrain)
AccuTest = neu_net.predict(DTest, LabelsTest)
print("Train Accuracy = {}%".format(round(AccuTrain,3)))
print("Test Accuracy = {}%".format(round(AccuTest,3)))

In [ ]:
'''Global Params'''
M = 100 #MiniBatch Size
n = len(DTrain[0])
nodesInHiddenLayers = [100,100]
r = totalClasses
neu_net = NeuralNetwork(nodesInHiddenLayers, "Sigmoid", "ReLU")
print(neu_net)
neu_net.fit(DTrain, oneHotTrainLables, eta=0.5, max_epoch=20000, epsilon=1e-16, adaptive=False)

print(neu_net.predict(DTrain, LabelsTrain))
print(neu_net.predict(DTest, LabelsTest))


In [149]:
def prediction(net, data, lbl):
    net.forwardFeed(data)
    outputs = net.layers[-1].output
    prediction = []
    for x in outputs:
        prediction.append(np.argmax(x))
    print(len(prediction), lbl.shape)
    accu_score = accuracy_score(lbl, prediction)
    print("Accuracy= ",accu_score)
prediction(neu_net, DTest, LabelsTest)

6500 (6500,)
Accuracy=  0.8349230769230769


In [42]:
#for layer in neu_net.layers:
neu_net.forwardFeed(DTrain)
layer = neu_net.layers[-1]
for x in layer.output:
    print(np.amax(x))

0.9574965608808638
0.9598815182687106
0.9715072274530074
0.9947492490063464
0.959231388849291
0.9951440532587261
0.9809080617540177
0.9896246420973588
0.9153667210798168
0.992969753786681
0.9981044205572329
0.9355051887869412
0.9921349059892898
0.8823832200701214
0.9954868734068857
0.9940287842713526
0.9633531984936602
0.9963201427429993
0.9986275947143104
0.9653733933674434
0.9831577060576818
0.9588271222967996
0.9885018466661959
0.9990097842251427
0.9968026686284973
0.15885816189179364
0.9984893360591487
0.9283550233150702
0.9772795114396674
0.18389109737959275
0.2966082890310889
0.9931268589655947
0.751149067838081
0.9779459661374764
0.9964525546707461
0.9962631601447332
0.964698660839957
0.9963507660837544
0.9454058016404548
0.9995183472195573
0.9895941014584699
0.9825490783943021
0.9786388723204049
0.9614511678844883
0.9790501218740826
0.9957116121237091
0.9734377517215334
0.9965624438421532
0.9257140835452006
0.7261369217014743
0.8848640039233276
0.9461636943625348
0.984298402532

0.999549824940158
0.9583165958115486
0.9866799711850157
0.9954757874882166
0.8209919122328607
0.9935633035149655
0.9622080587604429
0.9910711052765916
0.9657536665348124
0.9964553862764024
0.9930448563627859
0.9868115532092695
0.9775910383090698
0.9926318453886384
0.9588293652156331
0.11629515464341009
0.9973019900538026
0.9765974114038265
0.9289610468336756
0.9990300554766935
0.9784420205356531
0.9992865914888858
0.9870485008783597
0.9923298893538249
0.9515923003605694
0.9901749763772063
0.9275205299560039
0.984771314468094
0.9953843260607674
0.9364266426922657
0.15062100783431842
0.970622338008795
0.9983766533262356
0.9992176820538983
0.9937878321442973
0.9829620525574917
0.1135654778456229
0.960144465443138
0.989353530739217
0.8639473999625259
0.984598265674392
0.9342789374776173
0.9417658989267004
0.9866412731706173
0.9807626927990905
0.9422606103428718
0.996246208029141
0.9889280225572937
0.13188133230167118
0.9959688321900405
0.9930194064888934
0.10964032975712434
0.9974128300291

0.9990731454391749
0.9659882618966247
0.9960074275516607
0.941788362156794
0.8806603415516308
0.999433608616907
0.988799226310162
0.997724836239478
0.9907051255512678
0.9839870449749578
0.9588366385584915
0.9552442550777445
0.9982839733544133
0.9806800032018735
0.9189100100263156
0.9955767092835648
0.9786175599454197
0.9905559554797658
0.9926653951295042
0.9980756842379509
0.9974335310452737
0.9912363553624038
0.9664221347891023
0.986556859762788
0.9952408747688375
0.9317324055505014
0.9973078922339078
0.9872200716305767
0.9822929461817455
0.99219579397182
0.9981799922028751
0.9602638252823765
0.6544153487944524
0.9974157581490356
0.9816560264393934
0.9907132410282733
0.9788975153587594
0.08770192157611748
0.9751405426417964
0.9959457597321821
0.9900720570347098
0.9953509652393753
0.989150216773803
0.9813945340048795
0.9867300254350235
0.9779206639544359
0.9921601739304261
0.963887620438727
0.8609685539427605
0.9942537413224118
0.8665775140538479
0.8968082607672817
0.9981886375041656
0

0.9990730949771347
0.9880146117303191
0.993122656293516
0.913783075662218
0.9956693485377958
0.9268771624532435
0.9839268507685366
0.9432632973168988
0.987844288611153
0.9835620944720079
0.9378926014857123
0.9899260613644782
0.9860620751987377
0.9979369889013134
0.9798408597390246
0.9971340913261371
0.9774468953257035
0.9786680200654018
0.9982189957748637
0.991927029117033
0.9976569913295753
0.9990289910413533
0.9991257133261955
0.9714317418470496
0.9081945754896817
0.9922634270983182
0.9961309183425257
0.9835067857733272
0.9985426801314583
0.9926570391636708
0.9953906774275059
0.9746651351143472
0.9885423944649201
0.9994663548520822
0.9970000904155318
0.9965715090968659
0.9961422379580201
0.9730295350302943
0.9979966371980692
0.9986809752691265
0.9087113161341686
0.9961554614076766
0.9937275591396599
0.9950097849512352
0.9578521180319195
0.9824494865380219
0.9283690092877047
0.9966006760418895
0.9992164773293071
0.8899609913439677
0.9234809554164983
0.9896968321133269
0.99930723195611

0.9941010586540802
0.9175114092848669
0.9989185793534685
0.9601130687830255
0.9823969046675953
0.987509353371624
0.9750284497589784
0.7670713559762049
0.9799040026417628
0.9964602382043547
0.9805162162706789
0.9956329099297624
0.8233656894079155
0.9981591063364792
0.9871884825698406
0.9816209481238578
0.9864141583558487
0.955979372506749
0.9859988800344116
0.13401774928409046
0.9670118763841975
0.9982360549225116
0.9405168162505936
0.8449630772019121
0.9794755673928973
0.9970493614065721
0.9968950051556178
0.982654119449989
0.9793043544976379
0.9889850716476672
0.9835255763956517
0.9909452747110228
0.9726749526137094
0.9902044169400354
0.9822967651719073
0.9397496266431167
0.9953781176104015
0.98528014196879
0.9734367308613266
0.9905773548256758
0.9646235470298518
0.9971084881522744
0.9923935855161122
0.12179167227945877
0.989153750862932
0.9945707486649417
0.9783773448864858
0.9861900023542787
0.9955981666192218
0.9976374886402096
0.955883063250773
0.9946648338071928
0.991837933931166

0.9686872309284182
0.8865556418429398
0.8421657174750894
0.9970551624769926
0.9792267153941402
0.9312107936653224
0.988709275220367
0.9963267467649028
0.9732449156443563
0.9864819928941603
0.9949181214461918
0.9965197486381431
0.9997733337096967
0.8756943919813384
0.9962719634903471
0.9238871427957651
0.9773767457037114
0.991140746696942
0.8940525325226162
0.9524510235966921
0.9923652245801021
0.9126131870792538
0.9070937382745753
0.9952065661250453
0.9695186382711987
0.9965114926029761
0.9784354149918646
0.9956690147474752
0.9122721297761082
0.9962198598709128
0.9911658624517655
0.8704317268791218
0.9784968695377609
0.990981650400374
0.8865787515205397
0.9998296959803278
0.9777199672321042
0.9722076073938327
0.8954239769382363
0.9910896045165802
0.8930668742308524
0.9965093356471786
0.9707433237211693
0.9854465179162929
0.9978877644562963
0.9602459175248316
0.9528152054039902
0.9937953236224109
0.9771113624995028
0.9841301988728897
0.9994048730758934
0.9964187398556577
0.9956430318002

0.9762901335854246
0.993589867082027
0.9574000276922102
0.9969315201705081
0.9991391317326004
0.9584254482676492
0.9951239576899128
0.9802289002909856
0.9978646563943655
0.902844338024158
0.9204393974175842
0.9945744072795282
0.9965749409426169
0.9375218140509752
0.9875216340342119
0.9991951348714067
0.9980250314253161
0.9658595945943336
0.9736647223655758
0.9918174628168372
0.9417407567787459
0.9657012221860442
0.9901613272146944
0.9990725767218077
0.9623033604198427
0.9968760753243208
0.9993409955385026
0.892975828412651
0.9997083658579045
0.9184412237364045
0.9943538326342044
0.9898749745492691
0.9988382685344023
0.9368694817743144
0.9873821176357023
0.9706635020373873
0.9947454652241793
0.9982184234180854
0.6395931095227583
0.9639812667537268
0.8986431779014199
0.9798461901971136
0.9628626851285655
0.9875487116088311
0.9947461095425882
0.9132807678997429
0.9789439750042535
0.9989594740376052
0.9228554429165257
0.9903048483638562
0.6737868773195533
0.9126625549786438
0.9348472873365

0.9986757929158966
0.853851700257652
0.9933491723744412
0.9829139561242378
0.9757022817551907
0.9991522076246412
0.9038041678289573
0.8806411583441285
0.9973150267298406
0.9616134475577885
0.9954387725977478
0.9989809002648457
0.9451672643946626
0.9731763112716442
0.995935513310286
0.9975015278677118
0.9882740100824497
0.9589621454708124
0.8583023366420102
0.9838073875209323
0.9984758837364832
0.9397340978627425
0.9991475732368363
0.9301903966937753
0.9967872370284937
0.9557986563083808
0.9798198309730096
0.9958829513867713
0.952778940767526
0.9592058350565105
0.9026990913820084
0.9982497216892229
0.9957908402069714
0.9465362633443595
0.9815527459970046
0.981041591196775
0.9991915903044234
0.9889350198847466
0.6912174018282885
0.9424782372558494
0.9891607280232153
0.9984287212993815
0.9764646827743871
0.9898812422708924
0.9982099113642267
0.9837217879124529
0.9926662702876371
0.9901415483684962
0.9865095781577046
0.8710593952428433
0.9628263936077361
0.9955358433074734
0.99741369254876

0.9681237069858017
0.9030208255173512
0.9949707165661585
0.9316894094374559
0.999925469024291
0.9590455442898838
0.9969858588589979
0.9990838721116875
0.7539766940487614
0.9231015280154663
0.9966122888297829
0.9889047588763001
0.9898226453238298
0.3709102484704525
0.9988979244179339
0.7270107433816547
0.19606458605886548
0.16400794122895607
0.07665897738860282
0.05990250776918898
0.996570019240485
0.992515140099873
0.9245875291576124
0.9877528230939646
0.9891253546709208
0.9949599831511902
0.9927902520366166
0.9750683777419984
0.9893930525009575
0.6643527596049753
0.9462455529816627
0.996131648813837
0.9387047744017013
0.9926591391925069
0.9969796736543713
0.9289258560582272
0.9962308430245552
0.9663702286454984
0.9882633118085904
0.9869549553495592
0.9614995935319258
0.15417469885992616
0.9813887837510186
0.9535742245070195
0.9950218261947368
0.998045272855485
0.9719286997702826
0.9796425211525311
0.947937776499004
0.9839167075669587
0.9539194247224015
0.9940495737245869
0.20279554391

0.9632608445295217
0.9958370242106362
0.4129675851106628
0.9999041384302134
0.6115725780751997
0.9502094765933783
0.9941902502049107
0.9799596689766649
0.959199332439704
0.9926051078997304
0.9979116971319565
0.9984384040074931
0.894048703345617
0.9014036282217083
0.9834221093148212
0.7935252362851982
0.992191784779155
0.9991939995470803
0.9868080656822062
0.9816331437318814
0.9587417406752266
0.04830716319115995
0.9277416618514382
0.8567135785591287
0.9879198511925565
0.9532210456159614
0.9807989545000022
0.9974854393798459
0.9991143555835538
0.990885578674119
0.5213021799922649
0.9866264029095814
0.9857578169497314
0.9992359997852605
0.750224277264848
0.9970312931834135
0.9969680735480051
0.9924929204977738
0.9918205558025384
0.9973981844213454
0.9915893004597929
0.952849860065699
0.9501034112527961
0.9997070208357282
0.9880485790411746
0.9897176572232548
0.98783043655464
0.9955015604465235
0.9960835699074824
0.9784309942076175
0.9951811862487883
0.9814291386026751
0.9972627402063157


In [119]:
for layer in neu_net.layers:
    print(layer.weights[:20])

[[-0.00832659  0.00190687 -0.00842764 ... -0.00967168 -0.0031852
  -0.00788877]
 [-0.00374876 -0.00069765 -0.0061328  ... -0.00244248 -0.0087808
  -0.00771211]
 [-0.00667095 -0.00815066 -0.01345256 ... -0.01057041 -0.0077736
   0.00190876]
 ...
 [ 0.00256856 -0.00751513  0.00736599 ... -0.00118116  0.00596706
  -0.0104013 ]
 [-0.00783131  0.00732662 -0.01460057 ... -0.00683932 -0.0116679
  -0.01343878]
 [ 0.00912219 -0.00200028 -0.00287532 ... -0.00519311 -0.00377922
  -0.01066432]]
[[-6.21032252e-04 -2.59239947e-03 -9.56368099e-05 ...  4.04582347e-03
  -8.26309954e-03 -3.43393701e-03]
 [ 5.08014315e-03 -4.74109666e-03 -6.41263331e-03 ... -1.10355792e-02
  -4.81996743e-04  2.79888462e-03]
 [-2.70722512e-03 -4.15923573e-03 -6.37366078e-03 ... -5.13806128e-03
  -9.67501533e-03 -1.74358453e-02]
 ...
 [ 1.49045161e-03 -4.46847479e-03 -3.78802230e-04 ... -4.47119289e-03
  -8.11694549e-04 -4.70854976e-03]
 [-1.13828749e-03 -7.95371213e-04 -4.81715724e-03 ... -6.36309491e-03
  -1.28351958e-02

In [131]:
np.random.normal(0,0.00005, 10).reshape(5,2)

array([[-1.47126082e-05, -2.68311825e-05],
       [ 2.41010176e-05, -2.41943726e-06],
       [ 9.56130143e-05,  9.20987937e-06],
       [-4.36892046e-05, -1.35455218e-04],
       [-3.05009008e-05, -1.62088758e-07]])